# import modules

In [2]:
#1
!pip install -q -U json-lines

import json
import os
import pandas as pd
import tarfile
import json_lines
from tqdm.notebook import tqdm
from pprint import pprint
import glob
import matplotlib.pyplot as plt
import numpy as np

# read party, tweets, mentions and hashtags

In [3]:
#2
#funktioniert: get Party name + Tweets
files123 = glob.glob('./data/*.jl')

alles = []
partyname_n_tweets = []
partyname_mentions = []
partyname_hash = []
error = []

AfD = 0
Grünen = 0
CDU = 0
CSU = 0
Linke = 0
FDP = 0
SPD = 0
maxWert = 50000

for file in tqdm(files123): # responsible for visual output

    with json_lines.open(file) as f:
        for item in f:
            try:
                if item['response']['meta']['result_count'] == 0: 
                    continue
                elif item['account_data']['Partei'] == "Fraktionslos":
                    continue
                if item['http_status'] == 200:
                    
                    #get party name
                    partei = item["account_data"]["Partei"]
                    
                    #get all tweets per person
                    tweets_array = item["response"]["data"]
                    
                    for tweet in tweets_array: #range(0,len(tweets_array)-1):
                        hashtagArray = ""
                        
                       # print(type(tweet))
                        if("referenced_tweets" in tweet): # tweet is a dictionary
                            tweet_type = tweet["referenced_tweets"]
                            if(tweet_type[0]["type"] == "retweeted"):
                                continue
                        
                        if("entities" in tweet): 
                            entities = tweet["entities"]

                            if("hashtags" in entities):
                                hashtags = entities["hashtags"]
                                for tag in hashtags:
                                        hashtag = tag["tag"]
                                        hashtagArray = hashtagArray + " " + hashtag
                
                            if(not (hashtagArray == "")):
                        
                                if (partei == "AfD" and AfD < maxWert):
                                    AfD += 1
                                elif (partei == "Bündnis 90/Die Grünen" and Grünen < maxWert):
                                    Grünen += 1
                                elif (partei == "CDU" and CDU < maxWert):
                                    CDU += 1
                                elif (partei == "CSU" and CSU < maxWert):
                                    CSU += 1
                                elif (partei == "Die Linke" and Linke < maxWert):
                                    Linke += 1
                                elif (partei == "FDP" and FDP < maxWert):
                                    FDP += 1
                                elif (partei == "SPD" and SPD < maxWert):
                                    SPD += 1
                                else:
                                    continue
                                    
                                alles.append({"Partei": partei, "hashtags": hashtagArray})
                
                else:
                    error.append(item)
                    print("yikes")
                    
            except Exception as e:
                raise e
                           
print("Number of error-requests:", len(error))
print("Read", len(partyname_n_tweets), "Tweets sucussfully.")
print("mentions", len(partyname_mentions))
print("hashtags", len(partyname_hash))


Number of error-requests: 0
Read 0 Tweets sucussfully.
mentions 0
hashtags 0


# Data Preprocessing

In [4]:
#imports
import re
import nltk
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import Pipeline
from sklearn.feature_selection import SelectKBest, chi2
from sklearn.model_selection import train_test_split

#import ML LIB
from sklearn.svm import LinearSVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.neighbors import KNeighborsClassifier

#import Evaluation Metrics
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix

In [5]:
hash_df = pd.DataFrame(alles)
hash_df

,Partei,hashtags
0,Bündnis 90/Die Grünen,GreenDeal
1,Bündnis 90/Die Grünen,tagdermenschenrechte SchalfuersLeben
2,Bündnis 90/Die Grünen,BuildBackBetter Generaldebatte
3,Bündnis 90/Die Grünen,SachsenAnhalt Rundfunkbeitrag
4,Bündnis 90/Die Grünen,Rundfunkbeitrag SachsenAnhalt
...,...,...
323411,CSU,augsburg bahn elektrifizierung
323412,CSU,doenerverbot augsburg
323413,CSU,Chrisgruen doenerverbot augsburg
323414,CSU,doenerverbot augsburg stadtrat


In [6]:
hash_df["Partei"].value_counts()

Die Linke                50000
SPD                      50000
CDU                      50000
Bündnis 90/Die Grünen    50000
AfD                      50000
FDP                      50000
CSU                      23416
Name: Partei, dtype: int64

In [7]:
len(hash_df)

323416

In [8]:
#split data into train and test set
X_train, X_test, y_train, y_test = train_test_split(hash_df["hashtags"], hash_df.Partei, test_size = 0.3, random_state=111)

In [9]:
#X_train.value_counts()
X_train

202813                             fb
64871                     Zeitenwende
216135     Bankern Managern Klatschen
288892                            VDS
6065                  ldkbw Bundestag
                     ...             
105747                     weremember
102486         Belarus EU Lukaschenko
135892                       Dobrindt
10196           Lübeck antisemitismus
129900                            bvb
Name: hashtags, Length: 226391, dtype: object

In [10]:
y_train

202813                      CDU
64871                       AfD
216135                      SPD
288892                      FDP
6065      Bündnis 90/Die Grünen
                  ...          
105747                      SPD
102486                      FDP
135892                      CSU
10196                 Die Linke
129900    Bündnis 90/Die Grünen
Name: Partei, Length: 226391, dtype: object

In [22]:
stemmer = nltk.stem.SnowballStemmer("german")
nltk.download("stopwords")
stop_words=set(nltk.corpus.stopwords.words("german"))

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Runjie\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [19]:
#write our own tokenizer function
def tokenize(text):
    tokens = [word for word in nltk.word_tokenize(text)]
    tokens = map(str.lower, tokens)
    tokens = [word.strip(" ") for word in tokens]
    tokens = [item for item in tokens if (item not in stop_words)]
    return tokens

# Hashtags

## Support Vector

In [23]:
#ngram_range = how many words to consider, 1, 2 --> look at every seperate word, and a pair of 2 words: not good --> bad
pipeline = Pipeline([("vect", TfidfVectorizer(tokenizer=tokenize, ngram_range=(1, 2), stop_words=None, sublinear_tf=True, max_df=0.75, lowercase = False)), 
                     ("chi", SelectKBest(chi2, k=10000)),
                     ("clf", LinearSVC(C=1.0, penalty="l1", max_iter=6000, dual=False))])

In [24]:
model = pipeline.fit(X_train, y_train)

In [13]:
vectorizer = model.named_steps["vect"]
chi = model.named_steps["chi"]
clf = model.named_steps["clf"]

In [14]:
feature_names = vectorizer.get_feature_names()
feature_names = [feature_names[i] for i in chi.get_support(indices = True)]
feature_names = np.asarray(feature_names)

In [15]:
#target_names = ["Bündnis 90/Die Grünen", "SPD", "CDU", "Die Linke", "FDP", "AfD", "CSU"]
target_names = clf.classes_
print("top 30 keywords per class:")
for i, label in enumerate(target_names):
    top30 = np.argsort(clf.coef_[i])[-30:]
    print("%s: %s" % (label, " ".join(feature_names[top30])))
    print("\n")

top 30 keywords per class:
AfD: Frühsexualisierung Lückenpresse Bessermenschen NurnochAfD AfDwaehlen Grüninnen Kestner totalitär RemsMurr Kommunisten Spezialdemokraten WK193 Altparteien Pohl AfDimBundestag IntelligenzGibtsnichtimBioladen KrampfgegenRechts afdbayern Altparteienstaat2019 Ramelüringen pboehringer Klimawahn afdwirkt Brandnerunterwegs AfDwählen ARDZDF Staatsfunk nurnochAfD AfDwirkt Bernhard


Bündnis 90/Die Grünen: keinestartbahn3 Stilllovingfeminism SalonDerKinderrechte stilllovingfeminism NOAFD NoAFD WeAreInThisTogether Extremwetter derandereOsten Senden Schwabengrün ldkmv Chlorpyrifos bdk11 AImadeinEurope flucht HipHop DasandereSachsen cemtrail esc14 Cemtrail gefälltmirnicht Flächenfraß SHA ltmv Gentech gefaelltmirnicht ErneuerbareEnergien kreistagvr buergerschaftHST


CDU: Sénégal Ehrlichkeit FoRB Westerwald r2G Africa landesvater Landesvater cdulsa Untersuchungsausschuss Wirecard Heilmittelerbringer integrationjetzt nring Lichtenrade WK180 Remscheid wk180 le0101 nick20

In [26]:
prediction = model.predict(X_test)

In [27]:
print("******SVM classification report (hashtags, max 50k per party)******")
print("accuracy:", accuracy_score(y_test, prediction))
print(classification_report(y_test, prediction, target_names=target_names, digits=4))

******SVM classification report (hashtags, max 50k per party)******
accuracy: 0.5596598814738469
                       precision    recall  f1-score   support

                  AfD     0.7297    0.7068    0.7181     14975
Bündnis 90/Die Grünen     0.5733    0.5163    0.5433     15180
                  CDU     0.5471    0.3944    0.4583     14958
                  CSU     0.6663    0.4173    0.5132      6976
            Die Linke     0.7077    0.6239    0.6632     15075
                  FDP     0.6101    0.5290    0.5667     14963
                  SPD     0.3553    0.6543    0.4606     14898

             accuracy                         0.5597     97025
            macro avg     0.5985    0.5489    0.5605     97025
         weighted avg     0.5932    0.5597    0.5645     97025



In [28]:
print("************************SVM confusion matrix (hashtags, max 50k each party)************************")
cm = pd.DataFrame(
    confusion_matrix(y_test, prediction, labels=target_names),
    index = ["true:{:}".format(x) for x in target_names],
    columns = ["pred:{:}".format(x) for x in target_names]
)
cm.style.set_properties(**{'text-align': 'center'})

************************SVM confusion matrix (hashtags, max 50k each party)************************


,pred:AfD,pred:Bündnis 90/Die Grünen,pred:CDU,pred:CSU,pred:Die Linke,pred:FDP,pred:SPD
true:AfD,10585,584,672,91,388,736,1919
true:Bündnis 90/Die Grünen,670,7837,888,337,932,1023,3493
true:CDU,919,1338,5899,381,661,1204,4556
true:CSU,297,629,590,2911,323,616,1610
true:Die Linke,464,950,492,170,9405,548,3046
true:FDP,879,1125,1056,248,678,7916,3061
true:SPD,692,1207,1186,231,902,932,9748


## Decision Tree

In [29]:
#ngram_range = how many words to consider, 1, 2 --> look at every seperate word, and a pair of 2 words: not good --> bad
pipelineDT = Pipeline([("vect", TfidfVectorizer(ngram_range=(1, 2), stop_words=None, sublinear_tf=True, max_df=0.75, lowercase = False)), 
                     ("chi", SelectKBest(chi2, k=10000)),
                     ("clf", DecisionTreeClassifier(random_state = 111))])

In [30]:
modelDT = pipelineDT.fit(X_train, y_train)

In [31]:
vectorizerDT = modelDT.named_steps["vect"]
chiDT = modelDT.named_steps["chi"]
clfDT = modelDT.named_steps["clf"]

In [33]:
target_names = clfDT.classes_
print("leaves:", clfDT.get_n_leaves())
print("depth:", clfDT.get_depth())

leaves: 45146
depth: 9143


In [34]:
predictionDT = modelDT.predict(X_test)

In [35]:
print("***DecisionTree classification report (hashtags, max 50k)***")
print("accuracy:", accuracy_score(y_test, predictionDT))
print(classification_report(y_test, predictionDT, target_names=target_names, digits=4))

***DecisionTree classification report (hashtags, max 50k)***
accuracy: 0.5203195052821438
                       precision    recall  f1-score   support

                  AfD     0.7168    0.6616    0.6881     14975
Bündnis 90/Die Grünen     0.5296    0.4709    0.4985     15180
                  CDU     0.5121    0.3654    0.4265     14958
                  CSU     0.5489    0.3850    0.4526      6976
            Die Linke     0.7115    0.5681    0.6317     15075
                  FDP     0.5806    0.4742    0.5220     14963
                  SPD     0.3218    0.6455    0.4295     14898

             accuracy                         0.5203     97025
            macro avg     0.5602    0.5101    0.5213     97025
         weighted avg     0.5614    0.5203    0.5271     97025



In [36]:
print("*******************DecisionTree confusion matrix (hashtags, max 50k)*******************")
cm = pd.DataFrame(
    confusion_matrix(y_test, predictionDT, labels=target_names),
    index = ["true:{:}".format(x) for x in target_names],
    columns = ["pred:{:}".format(x) for x in target_names]
)
cm.style.set_properties(**{'text-align': 'center'})

*******************DecisionTree confusion matrix (hashtags, max 50k)*******************


,pred:AfD,pred:Bündnis 90/Die Grünen,pred:CDU,pred:CSU,pred:Die Linke,pred:FDP,pred:SPD
true:AfD,9907,683,787,193,352,833,2220
true:Bündnis 90/Die Grünen,777,7149,931,501,719,1039,4064
true:CDU,815,1382,5466,480,619,1140,5056
true:CSU,285,713,566,2686,302,550,1874
true:Die Linke,470,1063,624,248,8564,592,3514
true:FDP,858,1263,1153,377,681,7095,3536
true:SPD,709,1247,1146,408,800,971,9617


# RandomForest

In [54]:
#ngram_range = how many words to consider, 1, 2 --> look at every seperate word, and a pair of 2 words: not good --> bad
pipelineRF = Pipeline([("vect", TfidfVectorizer(ngram_range=(1, 2), stop_words=None, sublinear_tf=True, max_df=0.75, lowercase = False)), 
                     ("chi", SelectKBest(chi2, k=10000)),
                     ("clf", RandomForestClassifier(random_state = 111))])

In [55]:
modelRF = pipelineRF.fit(X_train, y_train)

In [56]:
vectorizerRF = modelRF.named_steps["vect"]
chiRF = modelRF.named_steps["chi"]
clfRF = modelRF.named_steps["clf"]

In [57]:
predictionRF = modelRF.predict(X_test)

In [58]:
target_names = clf.classes_

In [59]:
print("******RandomForest classification report (hashtags, max 50k)******")
print("accuracy:", accuracy_score(y_test, predictionRF))
print(classification_report(y_test, predictionRF, target_names=target_names, digits=4))

******RandomForest classification report (hashtags, max 50k)******
accuracy: 0.5317804689513013
                       precision    recall  f1-score   support

                  AfD     0.7532    0.6692    0.7087     14975
Bündnis 90/Die Grünen     0.5631    0.4704    0.5126     15180
                  CDU     0.5261    0.3785    0.4402     14958
                  CSU     0.5608    0.3926    0.4619      6976
            Die Linke     0.6980    0.5871    0.6378     15075
                  FDP     0.5884    0.4891    0.5342     14963
                  SPD     0.3258    0.6622    0.4367     14898

             accuracy                         0.5318     97025
            macro avg     0.5736    0.5213    0.5332     97025
         weighted avg     0.5750    0.5318    0.5392     97025



In [60]:
print("************************RandomForest confusion matrix (hashtags, max 50k)************************")
cm = pd.DataFrame(
    confusion_matrix(y_test, predictionRF, labels=target_names),
    index = ["true:{:}".format(x) for x in target_names],
    columns = ["pred:{:}".format(x) for x in target_names]
)
cm.style.set_properties(**{'text-align': 'center'})

************************RandomForest confusion matrix (hashtags, max 50k)************************


,pred:AfD,pred:Bündnis 90/Die Grünen,pred:CDU,pred:CSU,pred:Die Linke,pred:FDP,pred:SPD
true:AfD,10021,637,760,189,384,761,2223
true:Bündnis 90/Die Grünen,647,7141,912,479,836,1049,4116
true:CDU,702,1203,5661,487,706,1137,5062
true:CSU,234,623,567,2739,347,568,1898
true:Die Linke,393,880,596,221,8851,612,3522
true:FDP,752,1131,1122,354,694,7318,3592
true:SPD,555,1066,1142,415,863,992,9865


# Gradient Boosting

In [61]:
#ngram_range = how many words to consider, 1, 2 --> look at every seperate word, and a pair of 2 words: not good --> bad
pipelineGB = Pipeline([("vect", TfidfVectorizer(ngram_range=(1, 2), stop_words=None, sublinear_tf=True, max_df=0.75, lowercase = False)), 
                     ("chi", SelectKBest(chi2, k=10000)),
                     ("clf", GradientBoostingClassifier(random_state = 111))])

In [62]:
modelGB = pipelineGB.fit(X_train, y_train)

In [63]:
vectorizerGB = modelGB.named_steps["vect"]
chiGB = modelGB.named_steps["chi"]
clfGB = modelGB.named_steps["clf"]

In [64]:
predictionGB = modelGB.predict(X_test)

In [65]:
target_names = clfGB.classes_

In [66]:
print("******GradientBoosting classification report (hashtags, max 50k)******")
print("accuracy:", accuracy_score(y_test, predictionGB))
print(classification_report(y_test, predictionGB, target_names=target_names, digits=4))

******GradientBoosting classification report (hashtags, max 50k)******
accuracy: 0.3938160267972172
                       precision    recall  f1-score   support

                  AfD     0.8098    0.5601    0.6622     14975
Bündnis 90/Die Grünen     0.6112    0.2496    0.3545     15180
                  CDU     0.7012    0.1878    0.2962     14958
                  CSU     0.8038    0.2079    0.3303      6976
            Die Linke     0.8376    0.2589    0.3955     15075
                  FDP     0.6585    0.3097    0.4213     14963
                  SPD     0.2103    0.8886    0.3400     14898

             accuracy                         0.3938     97025
            macro avg     0.6618    0.3804    0.4000     97025
         weighted avg     0.6505    0.3938    0.4057     97025



In [67]:
print("************************GradientBoosing confusion matrix (hashtags, max 50k)************************")
cm = pd.DataFrame(
    confusion_matrix(y_test, predictionGB, labels=target_names),
    index = ["true:{:}".format(x) for x in target_names],
    columns = ["pred:{:}".format(x) for x in target_names]
)
cm.style.set_properties(**{'text-align': 'center'})

************************GradientBoosing confusion matrix (hashtags, max 50k)************************


,pred:AfD,pred:Bündnis 90/Die Grünen,pred:CDU,pred:CSU,pred:Die Linke,pred:FDP,pred:SPD
true:AfD,8387,304,261,16,84,280,5643
true:Bündnis 90/Die Grünen,386,3789,204,108,156,659,9878
true:CDU,419,455,2809,71,135,464,10605
true:CSU,131,416,98,1450,103,265,4513
true:Die Linke,233,366,117,40,3903,288,10128
true:FDP,447,508,225,54,138,4634,8957
true:SPD,354,361,292,65,141,447,13238


# KNN

In [48]:
#ngram_range = how many words to consider, 1, 2 --> look at every seperate word, and a pair of 2 words: not good --> bad
pipelineKNN = Pipeline([("vect", TfidfVectorizer(ngram_range=(1, 2), stop_words=None, sublinear_tf=True, max_df=0.75, lowercase = False)), 
                     ("chi", SelectKBest(chi2, k=10000)),
                     ("clf", KNeighborsClassifier())])

In [49]:
modelKNN = pipelineKNN.fit(X_train, y_train)

In [50]:
vectorizerKNN = modelKNN.named_steps["vect"]
chiKNN = modelKNN.named_steps["chi"]
clfKNN = modelKNN.named_steps["clf"]

In [51]:
predictionKNN = modelKNN.predict(X_test)

In [52]:
print("***************KNeighbors classification report (hashtags, max 50k)***************")
print("accuracy:", accuracy_score(y_test, predictionKNN))
print(classification_report(y_test, predictionKNN, target_names=target_names, digits=4))

***************KNeighbors classification report (hashtags, max 50k)***************
accuracy: 0.49612986343725846
                       precision    recall  f1-score   support

                  AfD     0.3356    0.7761    0.4686     14975
Bündnis 90/Die Grünen     0.5211    0.5018    0.5112     15180
                  CDU     0.5000    0.3898    0.4381     14958
                  CSU     0.5662    0.3813    0.4557      6976
            Die Linke     0.7283    0.5687    0.6387     15075
                  FDP     0.6073    0.4575    0.5219     14963
                  SPD     0.5957    0.3348    0.4287     14898

             accuracy                         0.4961     97025
            macro avg     0.5506    0.4871    0.4947     97025
         weighted avg     0.5494    0.4961    0.4982     97025



In [53]:
print("***KNeighbors confusion matrix (hashtags, max 50k)***")
cm = pd.DataFrame(
    confusion_matrix(y_test, predictionKNN, labels=target_names),
    index = ["true:{:}".format(x) for x in target_names],
    columns = ["pred:{:}".format(x) for x in target_names]
)
cm.style.set_properties(**{'text-align': 'center'})

***KNeighbors confusion matrix (hashtags, max 50k)***


,pred:AfD,pred:Bündnis 90/Die Grünen,pred:CDU,pred:CSU,pred:Die Linke,pred:FDP,pred:SPD
true:AfD,11622,851,814,168,337,704,479
true:Bündnis 90/Die Grünen,4023,7617,895,417,630,862,736
true:CDU,5012,1480,5831,409,543,933,750
true:CSU,1833,788,641,2660,265,527,262
true:Die Linke,3454,1148,600,250,8573,521,529
true:FDP,3825,1326,1298,416,623,6846,629
true:SPD,4861,1408,1584,378,800,879,4988


# Multilayer Perceptron Classifier

In [39]:
#ngram_range = how many words to consider, 1, 2 --> look at every seperate word, and a pair of 2 words: not good --> bad
pipelineMLP = Pipeline([("vect", TfidfVectorizer(ngram_range=(1, 2), stop_words=None, sublinear_tf=True, max_df=0.75, lowercase = False)), 
                     ("chi", SelectKBest(chi2, k=10000)),
                     #("clf", MLPClassifier(max_iter = 6000, random_state = 111))])
                     ("clf", MLPClassifier(hidden_layer_sizes=(7,7), max_iter = 6000, random_state = 111))])

In [40]:
modelMLP = pipelineMLP.fit(X_train, y_train)

In [41]:
vectorizerMLP = modelMLP.named_steps["vect"]
chiMLP = modelMLP.named_steps["chi"]
clfMLP = modelMLP.named_steps["clf"]

In [42]:
predictionMLP = modelMLP.predict(X_test)

In [43]:
print("number of layers:", clfMLP.n_layers_)

number of layers: 4


In [44]:
target_names = clfMLP.classes_

In [45]:
print("******Multilayer Perceptron classification report (hashtags, max 50k)******")
print("accuracy:", accuracy_score(y_test, predictionMLP))
print(classification_report(y_test, predictionMLP, target_names=target_names, digits=4))

******Multilayer Perceptron classification report (hashtags, max 50k)******
accuracy: 0.5473743880443185
                       precision    recall  f1-score   support

                  AfD     0.7323    0.7123    0.7221     14975
Bündnis 90/Die Grünen     0.5457    0.5238    0.5346     15180
                  CDU     0.3157    0.6487    0.4247     14958
                  CSU     0.6556    0.4022    0.4986      6976
            Die Linke     0.7533    0.5932    0.6637     15075
                  FDP     0.6099    0.5006    0.5498     14963
                  SPD     0.6366    0.3725    0.4700     14898

             accuracy                         0.5474     97025
            macro avg     0.6070    0.5362    0.5519     97025
         weighted avg     0.6030    0.5474    0.5565     97025



In [46]:
print("***Multilayer Perceptron (two hidden layer, each has 7 neurons) confusion matrix (hashtags, max 50k)***")
cm = pd.DataFrame(
    confusion_matrix(y_test, predictionMLP, labels=target_names),
    index = ["true:{:}".format(x) for x in target_names],
    columns = ["pred:{:}".format(x) for x in target_names]
)
cm.style.set_properties(**{'text-align': 'center'})

***Multilayer Perceptron (two hidden layer, each has 7 neurons) confusion matrix (hashtags, max 50k)***


,pred:AfD,pred:Bündnis 90/Die Grünen,pred:CDU,pred:CSU,pred:Die Linke,pred:FDP,pred:SPD
true:AfD,10666,615,2451,140,264,695,144
true:Bündnis 90/Die Grünen,669,7952,3874,369,650,960,706
true:CDU,907,1369,9703,316,526,1216,921
true:CSU,310,702,1962,2806,257,603,336
true:Die Linke,456,1203,3460,136,8943,380,497
true:FDP,900,1404,3868,245,492,7490,564
true:SPD,658,1326,5420,268,740,937,5549
